In [43]:
from crawl import Crawler
from parse import find_all_safely
import pandas as pd
import numpy as np
import os
from setup_db import DB, setup_db, cursor_as_df, execute_to_df

In [2]:
# db = setup_db()

In [3]:
path = "db/test_db.db"
db = DB(path)
db.run_setup_scripts()
db.insert_initial_data()


In [3]:
# db.close()

In [4]:
db.select_all("jobs")

,job_id,user_created,active,time_created,last_run,job_name
0,1,1,1,2019-10-10 00:00:00,None,test_job


In [5]:
db.select_all("requests")

,request_id,job_id,start_city,end_city,date
0,1,1,1915,1815,2020-01-02 00:00:00
1,2,1,1915,7588,2020-01-02 00:00:00
2,3,1,1915,7588,2020-01-03 00:00:00


In [264]:
db.show_tables()

[('users',),
 ('jobs',),
 ('requests',),
 ('distances',),
 ('results',),
 ('cities',),
 ('execution_logs',),
 ('working_days',)]

In [265]:
db.select_all("results")

,request_id,time_created,start_city,end_city,time,date,price,changes_number


SELECT 
request_id,
strftime('%d.%m.%Y',t2.time_created) as rideDate,
t2.start_city as departureCity,
t2.end_city as arrivalCity
FROM jobs as t1
left join requests t2 ON t1.job_id = t2.job_id
WHERE t1.active = 1


In [292]:
params_from_db = db.c.execute('''SELECT 
t2.request_id,
strftime("%d.%m.%Y",t2.date) as rideDate,
t2.start_city as departureCity,
t2.end_city as arrivalCity
FROM jobs as t1
left join requests as t2 ON t1.job_id = t2.job_id
WHERE t1.active = 1''')
colnames = [i[0] for i in params_from_db.description]
values = params_from_db.fetchall()

In [293]:
print(colnames)
print(values)

['request_id', 'rideDate', 'departureCity', 'arrivalCity']
[(1, '02.01.2020', 1915, 1815), (2, '02.01.2020', 1915, 7588), (3, '03.01.2020', 1915, 7588)]


In [294]:
constant_search_params = {
    "adult": "1",
    "_locale": "pl"
}


In [295]:
values

[(1, '02.01.2020', 1915, 1815),
 (2, '02.01.2020', 1915, 7588),
 (3, '03.01.2020', 1915, 7588)]

In [318]:
params_list = []
for single_params in values:
    params_list.append({"rideDate": single_params[1],
                       "departureCity": single_params[2],
                       "arrivalCity": single_params[3]})
    
params_list = [{**i, **constant_search_params} for i in params_list]
request_id_list = [i[0] for i in values]

[1, 2, 3]

In [299]:
params_list

[{'rideDate': '02.01.2020',
  'departureCity': 1915,
  'arrivalCity': 1815,
  'adult': '1',
  '_locale': 'pl'},
 {'rideDate': '02.01.2020',
  'departureCity': 1915,
  'arrivalCity': 7588,
  'adult': '1',
  '_locale': 'pl'},
 {'rideDate': '03.01.2020',
  'departureCity': 1915,
  'arrivalCity': 7588,
  'adult': '1',
  '_locale': 'pl'}]

In [319]:
current_params = params_list[1]
current_request_id = request_id_list[1]

In [320]:
crawler = Crawler(current_params)
crawler.get_all()
results_pd_raw  = crawler.get_pandas_df()

In [321]:
results_pd = results_pd_raw.copy()

In [322]:
results_pd_raw

,departure_time,arrival_time,departure_station,arrival_station,price
0,00:30,04:00,Kraków MDA,Łódź Fabryczna,"69,99"
1,07:05,11:25,Kraków MDA,Łódź Fabryczna,"69,99"
2,12:00,16:00,Kraków MDA,Łódź Fabryczna,"69,99"
3,15:10,19:25,Kraków MDA,Łódź Fabryczna,"69,99"
4,19:15,23:50,Kraków MDA,Łódź Fabryczna,None
5,23:59,04:15,Kraków MDA,Łódź Fabryczna,"45,99"
6,00:30,04:00,Kraków MDA,Łódź Fabryczna,"69,99"


In [350]:
class dfSanitizer:
    def __init__(self, df):
        self.df = df

#         self.sanitize_station_vec = np.vectorize(sanitize_station)
        self.sanitize_price_vec = np.vectorize(sanitize_price)
#         self.sanitize_time_vec = np.vectorize(sanitize_time)
    
    def sanitize_price(self, x):
        if x is None:
            return None

        try:
            stripped = re.sub("[^\d\,]", "", x) # remove all characters except numbers and commas

            if len(stripped)>10:
                return None

            stripped_comma = stripped.replace(",", ".")

            number = float(stripped_comma)

        except:
            return None

        return number
    

In [352]:
a = dfSanitizer(1)
a

In [353]:
a.sanitize_price_vec(10)

array(None, dtype=object)

In [323]:
def sanitize_price(x):
    if x is None:
        return None
    
    try:
        stripped = re.sub("[^\d\,]", "", x) # remove all characters except numbers and commas
        
        if len(stripped)>10:
            return None

        stripped_comma = stripped.replace(",", ".")
        
        number = float(stripped_comma)
    
    except:
        return None
    
    return number
    

In [324]:
print(sanitize_price("65,94") == 65.94)
print(sanitize_price("65,a94") == 65.94)
print(sanitize_price("65,a94   ") == 65.94)
print(sanitize_price("65,56,4") is None)
print(sanitize_price(None) is None)
print(sanitize_price("54,") == 54.0)
print(sanitize_price(",54") == 0.54)
print(sanitize_price(43) is None)

True
True
True
True
True
True
True
True


In [325]:
def sanitize_station(station):
    try:
        return station.lstrip().rstrip()
    except:
        return None

In [326]:
print(sanitize_station("abc") == "abc")
print(sanitize_station("      abc") == "abc")
print(sanitize_station("abc   ") == "abc")
print(sanitize_station(1) is None)
print(sanitize_station(None) is None)

True
True
True
True
True


In [327]:
def sanitize_time(x):
    if x is None:
        return None
    
    try:
        stripped = re.sub("[^\d\:]", "", x) # remove all characters except numbers and commas
        
        if len(stripped) != 5:
            return None

        if sum([i == ":" for i in stripped]) != 1:
            return None
        
    
    except:
        return None
    
    return stripped

In [328]:
print(sanitize_time("09:50") == "09:50")
print(sanitize_time("09:50   ") == "09:50")
print(sanitize_time("09:50aa") == "09:50")
print(sanitize_time("9:50:") is None)
print(sanitize_time("950") is None)
print(sanitize_time(":") is None)
print(sanitize_time("9:50") is None)

True
True
True
True
True
True
True


In [329]:

sanitize_station_vec = np.vectorize(sanitize_station)
sanitize_price_vec = np.vectorize(sanitize_price)
sanitize_time_vec = np.vectorize(sanitize_time)

In [330]:
results_pd.price = sanitize_price_vec(results_pd.price)
results_pd.departure_station = sanitize_station_vec(results_pd.departure_station)
results_pd.arrival_station = sanitize_station_vec(results_pd.arrival_station)
results_pd.departure_time = sanitize_time_vec(results_pd.departure_time)
results_pd.arrival_time = sanitize_time_vec(results_pd.arrival_time)
results_pd["fully_booked"] = results_pd.price.isna()

In [331]:
results_pd["rideDate"] = current_params["rideDate"]
results_pd["departure_datetime"] = results_pd.rideDate + " " + results_pd.departure_time
results_pd["departure_datetime"] = pd.to_datetime(results_pd["departure_datetime"])
results_pd["arrival_datetime"] = results_pd.rideDate + " " + results_pd.arrival_time
results_pd["arrival_datetime"] = pd.to_datetime(results_pd["arrival_datetime"])


In [332]:
mask = results_pd["arrival_datetime"] < results_pd["departure_datetime"]
masked_df = results_pd[mask]

In [333]:
results_pd.loc[mask, "arrival_datetime"] = masked_df.arrival_datetime + pd.DateOffset(days = 1)

In [339]:
from datetime import datetime

In [342]:
datetime.now()

datetime.datetime(2019, 12, 31, 13, 38, 0, 201930)

In [346]:
current_params

{'rideDate': '02.01.2020',
 'departureCity': 1915,
 'arrivalCity': 7588,
 'adult': '1',
 '_locale': 'pl'}

In [347]:
results_pd["request_id"] = current_request_id
results_pd["time_created"] = datetime.now()
results_pd["departure_city"] = current_params["departureCity"]
results_pd["arrival_city"] = current_params["arrivalCity"]

In [334]:
db.select_all("results")

,request_id,time_created,start_city,end_city,time,date,price,changes_number


In [349]:

results_pd

,departure_time,arrival_time,departure_station,arrival_station,price,fully_booked,rideDate,departure_datetime,arrival_datetime,request_id,time_created,departure_city,arrival_city
0,00:30,04:00,Kraków MDA,Łódź Fabryczna,69.99,False,02.01.2020,2020-02-01 00:30:00,2020-02-01 04:00:00,2,2019-12-31 13:39:37.571872,1915,7588
1,07:05,11:25,Kraków MDA,Łódź Fabryczna,69.99,False,02.01.2020,2020-02-01 07:05:00,2020-02-01 11:25:00,2,2019-12-31 13:39:37.571872,1915,7588
2,12:00,16:00,Kraków MDA,Łódź Fabryczna,69.99,False,02.01.2020,2020-02-01 12:00:00,2020-02-01 16:00:00,2,2019-12-31 13:39:37.571872,1915,7588
3,15:10,19:25,Kraków MDA,Łódź Fabryczna,69.99,False,02.01.2020,2020-02-01 15:10:00,2020-02-01 19:25:00,2,2019-12-31 13:39:37.571872,1915,7588
4,19:15,23:50,Kraków MDA,Łódź Fabryczna,NaN,True,02.01.2020,2020-02-01 19:15:00,2020-02-01 23:50:00,2,2019-12-31 13:39:37.571872,1915,7588
5,23:59,04:15,Kraków MDA,Łódź Fabryczna,45.99,False,02.01.2020,2020-02-01 23:59:00,2020-02-02 04:15:00,2,2019-12-31 13:39:37.571872,1915,7588
6,00:30,04:00,Kraków MDA,Łódź Fabryczna,69.99,False,02.01.2020,2020-02-01 00:30:00,2020-02-01 04:00:00,2,2019-12-31 13:39:37.571872,1915,7588


In [337]:
unique_stations = set(results_pd.departure_station.tolist() + results_pd.arrival_station.tolist())
unique_stations

{'Kraków MDA', 'Łódź Fabryczna'}

In [12]:
results = []
for search_params in params_list:
    crawler = Crawler(search_params)
    crawler.get_all()
    results.append(crawler.get_pandas_df())
    
results

[  departure_time arrival_time departure_station     arrival_station   price
 0          00:30        09:35        Kraków MDA              Gdańsk  129,99
 1          00:30        11:45        Kraków MDA  Gdańsk Hala Olivia  129,98,
   departure_time arrival_time departure_station arrival_station  price
 0          00:30        04:00        Kraków MDA  Łódź Fabryczna  69,99
 1          07:05        11:25        Kraków MDA  Łódź Fabryczna  69,99
 2          12:00        16:00        Kraków MDA  Łódź Fabryczna   None
 3          15:10        19:25        Kraków MDA  Łódź Fabryczna  69,99
 4          19:15        23:50        Kraków MDA  Łódź Fabryczna   None
 5          23:59        04:15        Kraków MDA  Łódź Fabryczna  45,99
 6          00:30        04:00        Kraków MDA  Łódź Fabryczna  69,99,
   departure_time arrival_time departure_station arrival_station  price
 0          00:30        04:00        Kraków MDA  Łódź Fabryczna  69,99
 1          07:05        11:25        Kraków MD